# Stitch outputs together
- Once you have fitted the weak, medium and dense diads, stitch them all together, and apply the Ne correction model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import DiadFit as pf
pf.__version__

In [2]:
import os
DayFolder=os.getcwd()
meta_path=DayFolder +  '\MetaData'
spectra_path=DayFolder + '\Spectra'

## Load in specra, and combine

In [3]:
import os.path
from os import path
if path.exists('Discarded_df.xlsx'):
    discard=pd.read_excel('Discarded_df.xlsx')
else:
    discard=None
if path.exists('Weak_Diads.xlsx'):
    grp1=pd.read_excel('Weak_Diads.xlsx')
else:
    grp1=None
if path.exists('Medium_Diads.xlsx'):
    grp2=pd.read_excel('Medium_Diads.xlsx')
else:
    grp2=None
if path.exists('Strong_Diads.xlsx'):
    grp3=pd.read_excel('Strong_Diads.xlsx')
else:
    grp3=None
df2=pd.concat([grp1, grp2, grp3], axis=0)
if discard is not None:
    discard_cols=discard[discard.columns.intersection(df2.columns)]
    df2=pd.concat([df2, discard_cols])

In [4]:
df2

,Unnamed: 0,filename,Splitting,Diad1_Combofit_Cent,Diad1_Combofit_Height,Diad1_Voigt_Cent,Diad1_Voigt_Area,Diad1_Voigt_Sigma,Diad1_Residual,Diad1_Prop_Lor,...,HB2_Sigma,C13_Cent,C13_Area,C13_Sigma,Diad2_Gauss_Cent,Diad2_Gauss_Area,Diad2_Gauss_Sigma,Diad1_Gauss_Cent,Diad1_Gauss_Area,Diad1_Gauss_Sigma
0,0,Cap19-05X1,102.805515,1286.621126,224.869979,1286.621176,337.974422,0.632937,1.712223,3.211265e-01,...,0.592090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,Cap19-05X2,102.759512,1286.620738,207.881721,1286.620788,295.229953,0.642750,2.221887,1.132358e-01,...,0.615225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,Cap19-06X1,102.977376,1286.142246,917.359408,1286.142296,1677.688283,0.722776,2.472196,4.923356e-01,...,0.858911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,Cap19-06X2,102.941174,1286.176076,1178.920275,1286.176126,2047.286829,0.703560,3.024881,4.267956e-01,...,0.747058,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,Cap19-07X1,102.735556,1286.700138,45.642775,1286.700138,69.634155,0.715356,1.685106,2.374417e-11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,0,POC5,104.649246,1282.314935,28702.142246,1282.314885,84821.844267,1.082584,32.126234,7.075983e-01,...,2.497954,1369.588312,1898.345984,0.501225,1385.888323,12727.433801,15.422324,1285.844393,8702.298969,12.747222
9,0,POC6,104.592181,1282.423044,27837.940211,1282.423094,82550.774309,1.087030,31.125207,7.036047e-01,...,2.399255,1369.590142,1850.567359,0.498119,1385.977783,12339.743865,16.073272,1282.865250,10167.330915,17.285718
10,0,POC7,104.528308,1282.556637,26116.385336,1282.556587,77687.358551,1.096680,34.341807,6.894321e-01,...,2.357545,1369.606318,1717.892911,0.494993,1384.961389,10576.933641,15.791623,1287.620727,7536.564062,13.355083
11,0,POC8,104.441217,1282.731559,14929.892944,1282.731609,44966.138827,1.107363,16.629497,6.926407e-01,...,2.270157,1369.629838,1005.867793,0.488770,1383.925199,5680.660090,16.087837,1282.436299,5051.014707,19.390664


## First, get the metadata for this day

In [5]:
# Put the common string you can in all your Ne lines here
ID_str='_'
file_ext_meta='txt' # Will only take files of this type 
diad_meta=pf.get_diad_files(path=meta_path,
file_ext='txt', exclude_str='N', 
            exclude_str_2='FG', 
            exclude_str_3='Cap',
                            exclude_str_4='window',
            exclude_type='.png',   sort=True)
diad_meta

['Cap19-05X1.txt',
 'Cap19-05X2.txt',
 'Cap19-06X1.txt',
 'Cap19-06X2.txt',
 'Cap19-07X1.txt',
 'Cap19-07X2.txt',
 'Cap19-08X1.txt',
 'Cap19-08X2.txt',
 'Cap19-12X1.txt',
 'Cap19-12X2.txt',
 'Cap19-17X1.txt',
 'Cap19-17X2.txt',
 'Cap19-18X1.txt',
 'Cap19-18X2.txt',
 'FG04_31_MI (1).txt',
 'FG04_31_MI.txt',
 'FG04_31_MI3 (1).txt',
 'FG04_31_MI3 (2).txt',
 'FG04_31_MI3.txt',
 'FG04_31_MI4 (1).txt',
 'FG04_31_MI4 (2).txt',
 'FG04_31_MI4.txt',
 'POC1.txt',
 'POC10.txt',
 'POC11.txt',
 'POC12.txt',
 'POC13.txt',
 'POC14.txt',
 'POC15.txt',
 'POC16.txt',
 'POC17.txt',
 'POC18.txt',
 'POC19.txt',
 'POC2.txt',
 'POC20.txt',
 'POC21.txt',
 'POC22.txt',
 'POC23.txt',
 'POC24.txt',
 'POC25.txt',
 'POC26.txt',
 'POC27.txt',
 'POC28.txt',
 'POC29.txt',
 'POC3.txt',
 'POC30.txt',
 'POC31.txt',
 'POC32.txt',
 'POC33.txt',
 'POC34.txt',
 'POC35.txt',
 'POC36.txt',
 'POC37.txt',
 'POC38.txt',
 'POC39.txt',
 'POC4.txt',
 'POC40.txt',
 'POC41.txt',
 'POC42.txt',
 'POC43.txt',
 'POC44.txt',
 'POC45.txt',


## Now get all the important information out of the metadata files

In [6]:
meta=pf.stitch_metadata_in_loop(path=meta_path, 
Allfiles=diad_meta, prefix=False,
trupower=False)
meta['filename'].iloc[0]

100%|██████████████████████████████████████████████████████████████████████████████████| 81/81 [00:02<00:00, 30.27it/s]

Done
Oof, try not to work after midnight!


'POC49.txt'

## Then get a simple file name you can stitch with the spectra

In [8]:
file_m=pf.extracting_filenames_generic(names=meta['filename'],
    prefix=False, str_prefix=" ",
   file_type='.txt')
# Checks they are all unique
file_m[0]

'POC49'

## Now get filename from spectra into same form as metadata

In [9]:
# Remove these to get the pure file name
file_s=pf.extracting_filenames_generic(
    prefix=False, str_prefix=" ",
    names=df2['filename'].reset_index(drop=True),
   file_type='.txt')

file_s

array(['Cap19-05X1', 'Cap19-05X2', 'Cap19-06X1', 'Cap19-06X2',
       'Cap19-07X1', 'Cap19-07X2', 'Cap19-08X1', 'Cap19-08X2',
       'Cap19-12X1', 'Cap19-12X2', 'Cap19-17X1', 'Cap19-17X2',
       'Cap19-18X1', 'Cap19-18X2', 'FG04_31_MI', 'FG04_31_MI4', 'POC25',
       'POC46', 'POC47', 'POC48', 'POC49', 'POC50 (Sub BG)', 'POC50',
       'POC51 (Sub BG)', 'POC51', 'POC52 (Sub BG)', 'POC52',
       'POC53 (Sub BG)', 'POC53', 'POC54 (Sub BG)', 'POC54',
       'POC55 (Sub BG)', 'POC55', 'POC56 (Sub BG)', 'POC56',
       'POC57-0.04 (Sub BG)', 'POC57-0.04', 'POC58 (Sub BG)', 'POC58',
       'FG04_31_MI3 (1)', 'FG04_31_MI3', 'FG04_31_MI4 (1)', 'POC10',
       'POC11', 'POC16', 'POC17', 'POC18', 'POC19', 'POC20', 'POC21',
       'POC22', 'POC23', 'POC24', 'POC26', 'POC27', 'POC28', 'POC29',
       'POC30', 'POC31', 'POC32', 'POC33', 'POC34', 'POC35', 'POC36',
       'POC37', 'POC38', 'POC39', 'POC40', 'POC41', 'POC42', 'POC43',
       'POC44', 'POC45', 'POC1', 'POC12', 'POC13', 'POC14', 'POC1

In [10]:
# Combining them together
meta['name_for_matching']=file_m
df2['name_for_matching']=file_s
df_combo=df2.merge(meta, on='name_for_matching')

In [11]:
from pickle import load
import pickle
import os
pickle_path=os.getcwd()
file_pkl=pickle_path + '/' + 'Ne_Line_Regression_Example_cali.pck'
with open(file_pkl, 'rb') as file_handle:
    my_loaded_model = pickle.load(file_handle)

In [12]:
df_combo['Ne_Correction']=my_loaded_model(df_combo['sec since midnight'])
df_combo.insert(1,'Corrected_Splitting', df_combo['Ne_Correction']*df_combo['Splitting'])

In [17]:
df_combo.to_excel('Calibration_fitting.xlsx')